In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# import tensorflow as tf
# # Get the GPU device name.
# device_name = tf.test.gpu_device_name()

# # The device name should look like the following:
# if device_name == '/device:GPU:0':
#     print('Found GPU at: {}'.format(device_name))
# else:
#     raise SystemError('GPU device not found')


In [ ]:
import torch


if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 4.8MB/s 
     |████████████████████████████████| 890kB 40.6MB/s 
     |████████████████████████████████| 1.1MB 37.6MB/s 
     |████████████████████████████████| 3.0MB 53.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=4cde702c1992828d5b544548d622433032479cde3c4b51bfb71ed52e9a1dbfb8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

%matplotlib inline
import pandas as pd
df = pd.read_csv('/content/dataset_main_train.csv')
df.head(10)

,dataset,sentiment
0,environmental footprint hardware infrastructur...,1
1,directly produced renewable energy purchased p...,1
2,regulations operates legal definition fresh wa...,1
3,concerns leading increased regulatory scrutiny...,1
4,data used internally circumstances shares sell...,1
5,purposes user information includes information...,1
6,liabilities opposing party others whether resu...,1
7,records b information linked linkable individu...,1
8,overriding service provision indirectly requir...,1
9,cyber attacks social engineering threats puts ...,1


In [ ]:

df['sentiment'] = df['sentiment'].astype(int) 

df.head(10)

,dataset,sentiment
0,environmental footprint hardware infrastructur...,1
1,directly produced renewable energy purchased p...,1
2,regulations operates legal definition fresh wa...,1
3,concerns leading increased regulatory scrutiny...,1
4,data used internally circumstances shares sell...,1
5,purposes user information includes information...,1
6,liabilities opposing party others whether resu...,1
7,records b information linked linkable individu...,1
8,overriding service provision indirectly requir...,1
9,cyber attacks social engineering threats puts ...,1


In [ ]:
from sklearn.model_selection import train_test_split

train_inputs, valid_inputs, train_labels, valid_labels = train_test_split(df.dataset	, df.sentiment, random_state=777, test_size=0.2)
train_inputs, train_labels

(3426    ry color customer basis helpful trader perspec...
 3500    mentioned growth asset fee base account partia...
 4672    ration grow dollar term finally annual depreci...
 6687    Mueller Water Products SA Earnings Summary Pre...
 8031    low IFRS implementation second net exchange ga...
                               ...                        
 985     ovide great value deliver exceptional service ...
 5789    ke think look like couple year fully accretive...
 4017    e maybe Jens force majeure delicate topic mayb...
 7767    x rate company include potential impact change...
 2982     Zachary Evershed National Bank Financial line...
 Name: dataset, Length: 7728, dtype: object, 3426    0
 3500    0
 4672    0
 6687    0
 8031    0
        ..
 985     0
 5789    0
 4017    0
 7767    0
 2982    0
 Name: sentiment, Length: 7728, dtype: int64)

In [ ]:
from transformers import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
def data_process(inputs, labels, batch_size=32): 
  label_list = ['0', '1']
  examples = [InputExample(str(i), t, label=str(labels.iloc[i])) for i, t in enumerate(inputs)]
  features = glue_convert_examples_to_features(examples, tokenizer, max_length =64,  label_list = label_list, output_mode='classification')

  all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
  all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
  all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
  all_labels = torch.tensor([f.label for f in features], dtype=torch.long)

  # Create the DataLoader for our training set.
  dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
  sampler = RandomSampler(dataset)
  batch_size = 32
  dataloader = DataLoader(dataset, sampler=sampler, batch_size=batch_size)
  return dataloader

In [ ]:
train_dataloader = data_process(train_inputs, train_labels)
valid_dataloader = data_process(valid_inputs, valid_labels)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    
    
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 3)

# run this model on the GPU. 
model.cuda()

# optimizer
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5,
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from transformers import get_linear_schedule_with_warmup


epochs = 20


total_steps = len(train_dataloader) * epochs


scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_token_type_ids = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, # b_token_type_ids
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    #print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in valid_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=b_token_type_ids, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")

======== Epoch 1 / 20 ========
Training...
  Batch    40  of    242.    Elapsed: 0:00:14.
  Batch    80  of    242.    Elapsed: 0:00:28.
  Batch   120  of    242.    Elapsed: 0:00:42.
  Batch   160  of    242.    Elapsed: 0:00:56.
  Batch   200  of    242.    Elapsed: 0:01:11.
  Batch   240  of    242.    Elapsed: 0:01:25.
  Average training loss: 0.03
  Training epcoh took: 0:01:26
Running Validation...
  Accuracy: 1.00
  Validation took: 0:00:08
======== Epoch 2 / 20 ========
Training...
  Batch    40  of    242.    Elapsed: 0:00:15.
  Batch    80  of    242.    Elapsed: 0:00:30.
  Batch   120  of    242.    Elapsed: 0:00:46.
  Batch   160  of    242.    Elapsed: 0:01:02.
  Batch   200  of    242.    Elapsed: 0:01:18.
  Batch   240  of    242.    Elapsed: 0:01:34.
  Average training loss: 0.00
  Training epcoh took: 0:01:34
Running Validation...
  Accuracy: 1.00
  Validation took: 0:00:08
======== Epoch 3 / 20 ========
Training...
  Batch    40  of    242.    Elapsed: 0:00:16.
  Batc

In [ ]:
#testing
text='Stock raise highly and the company gets big revenue this quarter.'
def predict(text, model=model):
  inputs_1 = tokenizer.encode_plus(text, max_length=512, add_special_tokens=True, return_tensors='pt')
  pred = model(inputs_1['input_ids'].to(device), token_type_ids=inputs_1['token_type_ids'].to(device), attention_mask=inputs_1['attention_mask'].to(device))[0].argmax().item()
  return pred

predict(text)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0

In [35]:
from sklearn.metrics import accuracy_score, confusion_matrix
from collections import Counter
df= df[:50]
df['pred'] = df['dataset'].apply(predict)
print(accuracy_score(df.sentiment, df.pred))
print(Counter(df.pred))


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

1.0
Counter({1: 50})


In [40]:
'''import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
cm = confusion_matrix(df.sentiment, df.pred, labels=[0,1])
labels=[0,1]
df_cm = pd.DataFrame(cm, index = [i for i in labels],
                  columns = [i for i in labels])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, cmap="Blues")'''

'import seaborn as sn\nimport pandas as pd\nimport matplotlib.pyplot as plt\ncm = confusion_matrix(df.sentiment, df.pred, labels=[0,1])\nlabels=[0,1]\ndf_cm = pd.DataFrame(cm, index = [i for i in labels],\n                  columns = [i for i in labels])\nplt.figure(figsize = (10,7))\nsn.heatmap(df_cm, annot=True, cmap="Blues")'

In [ ]:
pred = []
count = 0
for text in df_unlabel.text_no_stopwords:
  if count % 5000 == 0:
    print(f'finish {count}')
  cur_pred = predict(text)
  pred.append(cur_pred)
  count += 1

In [ ]:
from collections import Counter
Counter(pred)

In [ ]:
df_unlabel['pred'] = pred
df_unlabel

Combine predcitons and labeled samples

In [ ]:
 pred = df_unlabel.reset_index()
pred = pred[['news_id','pred']]
pred = pred.rename(columns={'pred':'label'})
pred